# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2025-12-30 08:43:18] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-12-30 08:43:18] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-12-30 08:43:18] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-12-30 08:43:20] INFO server_args.py:1565: Attention backend not specified. Use fa3 backend by default.


[2025-12-30 08:43:20] INFO server_args.py:2443: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=128 avail_mem=10.44 GB):   0%|          | 0/20 [00:00<?, ?it/s]

Capturing batches (bs=80 avail_mem=9.47 GB):  20%|██        | 4/20 [00:00<00:01, 14.25it/s]

Capturing batches (bs=40 avail_mem=9.45 GB):  50%|█████     | 10/20 [00:00<00:00, 21.74it/s]

Capturing batches (bs=8 avail_mem=5.81 GB):  80%|████████  | 16/20 [00:00<00:00, 20.85it/s] 

Capturing batches (bs=1 avail_mem=5.53 GB): 100%|██████████| 20/20 [00:01<00:00, 19.97it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Mafa. I'm from Brazil, and I'm a huge fan of Brazilian music. I'm currently a singer-songwriter. I want to tell you a story of my life so that you can understand me better.
My childhood was filled with the joy of music and dancing. I grew up in a family that loved music and danced to the beat of the music. I loved Brazilian music and dancing, and I was always eager to learn.
I started singing in church when I was a child, and I've been singing in churches ever since. I've been a member of a small Brazilian choir for many years. I've also been
Prompt: The president of the United States is
Generated text:  a man. He is not a woman. The president of the United States is a man. He is not a woman. What does the president of the United States have? (1 point) 
A. A job
B. A job as a man
C. A job as a woman
D. A job as a man or a woman
Answer: D

Is the following statement true or false: The president of the United States is a male. (1 point) A. True 

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [Age] year old [Occupation]. I'm a [Type of Vehicle] [Vehicle Name]. I'm a [Favorite Hobby] [Favorite Activity]. I'm a [Favorite Book/TV Show/Artist]. I'm a [Favorite Person]. I'm a [Favorite Place]. I'm a [Favorite Food]. I'm a [Favorite Sport]. I'm a [Favorite Movie]. I'm a [Favorite Music Artist]. I'm a [Favorite Animal]. I'm a [Favorite Book/TV Show/Artist]. I'm a [Favorite Person]. I'm a [Favorite Place]. I

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, the city known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It is also the seat of the French government and the country's cultural and political center. Paris is a vibrant and diverse city with a rich history and a strong sense of French identity. It is a popular tourist destination and a major economic hub in Europe. The city is home to many famous French artists, writers, and musicians, and is known for its cuisine, fashion, and wine. Paris is a city of contrasts, with its modern architecture and historical landmarks blending seamlessly into one another. It is

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by several key trends:

1. Increased integration with human intelligence: AI is likely to become more integrated with human intelligence, allowing machines to learn from and adapt to human behavior and decision-making processes.

2. Enhanced creativity and innovation: AI is likely to become more capable of generating creative and innovative ideas, as well as performing tasks that require human creativity and ingenuity.

3. Improved ethical and responsible AI: As AI becomes more integrated into our daily lives, there will be a greater emphasis on ethical and responsible AI development, with a focus on minimizing harm and maximizing benefits.

4. Increased use of AI in healthcare: AI



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  _______ and I am a/an _________. Nice to meet you. I have always loved to be outdoors, and it is something that I enjoy very much. I have a desire to travel and explore the world. I am a natural adventurer and enjoy trying new things. I am also a good listener and have a natural ability to understand people. I am always open to new experiences and have a deep respect for the things that matter most. I love to share my ideas and passions with others and am always looking for ways to make the world a better place. 

This is a short, neutral self-introduction, it seems like it

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, also known as the City of Light and the City of Many Faces. The city is located on the Seine River and is home to many cultural institutions, museums, and historical landmarks. It has 

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

First

 Name

]

...

 to

 achieve

 manually

.



3

In [6]:
llm.shutdown()